# Маленькое задание на pandas:

файл trust.csv - это результаты расчетов одного из наших скриптов,

задача кластеризировать эти результаты

**BalanceID** - номер балансовой группы, для которой мы проводили расчет (он уникальный)

**DateMonth** - месяц, по которому произведен расчет

**DateYear** - год, по которому произведен расчет

**PercentTransmissionPU** - процент передачи показаний от приборов учета

**IndexComplianceForecastPresentUnbalance** - индекс соответствия спрогонозированного и реального небаланса

**TrustIndexPSKfiz** - индекс доверия к данным ПСК физ лица

**TrustIndexPSKODN** - индекс доверия к данным ПСК ОДН

**TrustIndexPSKurik** - индекс доверия к данным ПСК юридические лица

**GUID** - технический id (его не трогать)

In [75]:
import pandas as pd

In [76]:
trust = pd.read_csv('trust.csv')

# 1ый алгоритм:

**первая группа** - 3 индекса доверия (физ, юрид, ОДН) находятся в интервале от 100 до 75% включительно

**вторая** - от 74 до 50% включительно

**третья** - от 49 до 1% включительно

**четвретая** - все что больше 101% и меньше 0%

сохранить результаты

In [89]:
first_group = trust[(trust['TrustIndexPSKfiz'] >= 75.) & (trust['TrustIndexPSKfiz'] <= 100.) & (trust['TrustIndexPSKODN'] >= 75.) & (trust['TrustIndexPSKODN'] <= 100.) & (trust['TrustIndexPSKurik'] >= 75.) & (trust['TrustIndexPSKurik'] <= 100.)]
if len(first_group) != 0:
    first_group.to_csv('first_group.csv', index=False)

In [90]:
second_group = trust[(trust['TrustIndexPSKfiz'] >= 50.) & (trust['TrustIndexPSKfiz'] <= 74.) & (trust['TrustIndexPSKODN'] >= 50.) & (trust['TrustIndexPSKODN'] <= 74.) & (trust['TrustIndexPSKurik'] >= 50.) & (trust['TrustIndexPSKurik'] <= 74.)]
if len(second_group) != 0:
    second_group.to_csv('second_group.csv', index=False)

In [91]:
third_group = trust[(trust['TrustIndexPSKfiz'] >= 1.) & (trust['TrustIndexPSKfiz'] <= 49.) & (trust['TrustIndexPSKODN'] >= 1.) & (trust['TrustIndexPSKODN'] <= 49.) & (trust['TrustIndexPSKurik'] >= 1.) & (trust['TrustIndexPSKurik'] <= 49.)]
if len(third_group) != 0:
    third_group.to_csv('third_group.csv', index=False)

In [92]:
fourth_group = trust[((trust['TrustIndexPSKfiz'] >= 101.) | (trust['TrustIndexPSKfiz'] <= 0.)) & ((trust['TrustIndexPSKODN'] >= 101.) | (trust['TrustIndexPSKODN'] <= 0.)) & ((trust['TrustIndexPSKurik'] >= 101.) | (trust['TrustIndexPSKurik'] <= 0.))]
if len(fourth_group) != 0:
    fourth_group.to_csv('fourth_group.csv', index=False)

# 2ой алгоритм:

Если **хотя бы один из индексов доверия превышал 100% дважды за все время наблюдения** (т.е. во всей выборке), то сохранять **BalanceID**, **месяцы** (в которые было превышение), **значение индексов** (которые повторялись)

In [83]:
#Создаю словарьи из индексов превышающие значение 101
d_fiz = {}
d_odn = {}
d_ur = {}
for i in range(len(trust)):
    if trust.TrustIndexPSKfiz[i] > 101 and trust.TrustIndexPSKfiz[i] in d_fiz.keys():
        d_fiz[trust.TrustIndexPSKfiz[i]] += 1
    elif trust.TrustIndexPSKfiz[i] > 101:
        d_fiz.update({trust.TrustIndexPSKfiz[i]: 1})
    if trust.TrustIndexPSKODN[i] > 101 and trust.TrustIndexPSKODN[i] in d_odn.keys():
        d_odn[trust.TrustIndexPSKODN[i]] += 1
    elif trust.TrustIndexPSKODN[i] > 101:
        d_odn.update({trust.TrustIndexPSKODN[i]: 1})
    if trust.TrustIndexPSKurik[i] > 101 and trust.TrustIndexPSKurik[i] in d_ur.keys():
        d_ur[trust.TrustIndexPSKurik[i]] += 1
    elif trust.TrustIndexPSKurik[i] > 101:
        d_ur.update({trust.TrustIndexPSKurik[i]: 1})

In [93]:
ans = pd.DataFrame()

In [94]:
#Добавляем в новую таблицу BalanceId, DateMonth, Index, когда значение в словарях больше 1 (когда больше 1, значит 
#эти индексы повторялись)
for i in range(len(trust)):
    if trust.TrustIndexPSKfiz[i] in d_fiz.keys() and d_fiz[trust.TrustIndexPSKfiz[i]] > 1:
        line = {'BalanceId': trust.BalanceId[i], 'DateMonth': trust.DateMonth[i], 'Index': trust.TrustIndexPSKfiz[i]}
        ans = ans.append(line, ignore_index=True)
    if trust.TrustIndexPSKODN[i] in d_odn.keys() and d_odn[trust.TrustIndexPSKODN[i]] > 1:
        line = {'BalanceId': trust.BalanceId[i], 'DateMonth': trust.DateMonth[i], 'Index': trust.TrustIndexPSKODN[i]}
        ans = ans.append(line, ignore_index=True)
    if trust.TrustIndexPSKurik[i] in d_ur.keys() and d_ur[trust.TrustIndexPSKurik[i]] > 1:
        line = {'BalanceId': trust.BalanceId[i], 'DateMonth': trust.DateMonth[i], 'Index': trust.TrustIndexPSKurik[i]}
        ans = ans.append(line, ignore_index=True)

In [95]:
#Сортирую словарь по индексам, чтобы легче было воспринимать повторяющиеся значения, далее записываю таблицу в файл
ans = ans.sort_values('Index')
ans.reset_index(drop=True, inplace=True)
ans.to_csv('task2.csv', index=False)

In [96]:
ans

,BalanceId,DateMonth,Index
0,2917.0,9.0,104.0
1,3254.0,9.0,104.0
2,2433.0,8.0,105.5
3,1278.0,8.0,105.5
4,3409.0,8.0,106.0
...,...,...,...
745,3474.0,8.0,1741.0
746,1308.0,8.0,1741.0
747,1703.0,8.0,3710.5
748,3242.0,8.0,3710.5
